In [1]:
import pandas as pd 
import numpy as np
import pickle
import os

In [2]:
with open("/home/naradaw/dev/Charles_Schwab/data/symbol_universe/snp_unique_100_2019", "rb") as fp:
    symbol_universe = pickle.load(fp)
    
symbol_universe = symbol_universe[:20]

In [3]:
print(symbol_universe)

['SWKS', 'ALLE', 'BDX', 'CMI', 'APH', 'PNC', 'GWW', 'GLW', 'NRG', 'JKHY', 'LRCX', 'KHC', 'OMC', 'KMX', 'MRO', 'GE', 'NVDA', 'NOC', 'KO', 'CE']


In [4]:
data_path = "/home/naradaw/dev/Charles_Schwab/data/w_features/v1/2024_10_31/dataset_sqs_2024_10_31_11_18.pkl"

In [5]:
with open(data_path, 'rb') as f:
    data = pickle.load(f)

features = np.array([data[symbol]['features'] for symbol in symbol_universe])
returns = np.array([data[symbol]['returns'] for symbol in symbol_universe])

In [6]:
del data

In [7]:
features.shape, returns.shape

((20, 1174, 60, 87), (20, 1174))

In [8]:
features = features.transpose(1,2,0,3)
returns = returns.transpose(1,0)

In [9]:
features.shape, returns.shape

((1174, 60, 20, 87), (1174, 20))

In [10]:
returns

array([[-0.12046958, -0.08060968, -0.13127704, ..., -0.03755041,
         0.00137822, -0.08290856],
       [-0.17279074, -0.0797321 , -0.14632595, ..., -0.10558641,
        -0.03582677, -0.13263543],
       [-0.18360105, -0.09518459, -0.05204666, ..., -0.09238129,
        -0.06323645, -0.15248878],
       ...,
       [-0.02662752, -0.02339783, -0.00908337, ..., -0.03014752,
        -0.07222377, -0.04400562],
       [ 0.0272451 , -0.03580051, -0.01721356, ..., -0.07323307,
        -0.0857621 , -0.06025173],
       [-0.07574979, -0.04655787, -0.01313814, ..., -0.0622228 ,
        -0.07168005, -0.05300828]])